In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble


24/02/27 14:00:41 WARN Utils: Your hostname, AspireA51556 resolves to a loopback address: 127.0.1.1; using 172.31.241.142 instead (on interface eth0)
24/02/27 14:00:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/27 14:00:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/27 14:00:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
df = spark.read.parquet('code/data/fhv_tripdata_2019-10.parquet')

In [6]:
df.repartition(6).write.parquet("code/pq/fhv/2019/10", mode='overwrite')

In [7]:
df = spark.read.parquet("code/pq/fhv/2019/10")

In [8]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropOff_datetime',
 'PUlocationID',
 'DOlocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [9]:
df.registerTempTable('fhv')

/home/pavlo/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [11]:
spark.sql("""
SELECT
      date_trunc('day', pickup_datetime) AS pickup_day, 
    count(1) count_
FROM
    fhv
GROUP BY 
    date_trunc('day', pickup_datetime)
order  BY 
    date_trunc('day', pickup_datetime)          
""").show()

+-------------------+------+
|         pickup_day|count_|
+-------------------+------+
|2019-09-30 00:00:00|  7430|
|2019-10-01 00:00:00| 60263|
|2019-10-02 00:00:00| 69378|
|2019-10-03 00:00:00| 71503|
|2019-10-04 00:00:00| 68816|
|2019-10-05 00:00:00| 52365|
|2019-10-06 00:00:00| 45090|
|2019-10-07 00:00:00| 65876|
|2019-10-08 00:00:00| 63540|
|2019-10-09 00:00:00| 61060|
|2019-10-10 00:00:00| 69083|
|2019-10-11 00:00:00| 67712|
|2019-10-12 00:00:00| 52189|
|2019-10-13 00:00:00| 44486|
|2019-10-14 00:00:00| 52457|
|2019-10-15 00:00:00| 63154|
|2019-10-16 00:00:00| 68172|
|2019-10-17 00:00:00| 68280|
|2019-10-18 00:00:00| 68440|
|2019-10-19 00:00:00| 53371|
+-------------------+------+
only showing top 20 rows



In [14]:
spark.sql("""
SELECT
 max( bigint(to_timestamp(dropOff_datetime)) -
          bigint(to_timestamp(pickup_datetime)))/60 longest_trip
FROM
    fhv     
""").show()

+------------+
|longest_trip|
+------------+
|  3.786915E7|
+------------+



In [15]:
dfz = (
    spark.read.option("header", "true")
    .csv("code/data/taxi_zone_lookup.csv")
)

In [16]:
dfz.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [30]:
dfz.registerTempTable('zones')


/home/pavlo/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


ModuleNotFoundError: No module named 'org'

In [37]:
dfr = spark.sql(""" SELECT
    Zone, 
    count(1) count_
FROM
    fhv t join zones z on t.PUlocationID = z.LocationID
GROUP BY 
    Zone
ORDER BY  count_
""").show()
# dfr.toPandas().sort_values(['count_'],)
# dfr.toPandas().sort_values(['count_'],)

+--------------------+------+
|                Zone|count_|
+--------------------+------+
|         Jamaica Bay|     1|
|Governor's Island...|     2|
| Green-Wood Cemetery|     5|
|       Broad Channel|     8|
|     Highbridge Park|    14|
|        Battery Park|    15|
|Saint Michaels Ce...|    23|
|Breezy Point/Fort...|    25|
|Marine Park/Floyd...|    26|
|        Astoria Park|    29|
|    Inwood Hill Park|    39|
|       Willets Point|    47|
|Forest Park/Highl...|    53|
|  Brooklyn Navy Yard|    57|
|        Crotona Park|    62|
|        Country Club|    77|
|     Freshkills Park|    89|
|       Prospect Park|    98|
|     Columbia Street|   105|
|  South Williamsburg|   110|
+--------------------+------+
only showing top 20 rows

